In [2]:
import gplace

location = "13.744677,100.5295593"  # Latitude and Longitude
keyword = "ร้านกาแฟ"
result = gplace.nearby_search(keyword, location)

In [3]:
from typing import TypedDict, Optional

class NearbyDenseCommunityInput(TypedDict):
    location_name: str
    radius: int

In [4]:
def find_place_from_text(location:str):
    """Finds a place and related data from the query text"""
    
    result = gplace.find_place_from_text(location)
    r = result['candidates'][0]
    return f"""
    address: {r['formatted_address']}\n
    location: {r['geometry']['location']}\n
    name: {r['name']}\n
    opening hours: {r['opening_hours']}\n
    rating: {r['rating']}\n
    """
    
def nearby_search(keyword:str, location:str, radius=2000, place_type=None):
    """Searches for many places nearby the location based on a keyword. using keyword like \"coffee shop\", \"restaurants\". radius is the range to search from the location"""
    location = gplace.find_location(location, radius=radius)
    result = gplace.nearby_search(keyword, location, radius)
    
    strout = ""
    for r in result:
        # Use .get() to handle missing keys
        address = r.get('vicinity', 'N/A')
        location_info = r.get('geometry', {}).get('location', 'N/A')
        name = r.get('name', 'N/A')
        opening_hours = r.get('opening_hours', 'N/A')
        rating = r.get('rating', 'N/A')
        plus_code = r.get('plus_code', {}).get('global_code', 'N/A')
        
        strout += f"""
        address: {address}\n
        location: {location_info}\n
        name: {name}\n
        opening hours: {opening_hours}\n
        rating: {rating}\n
        plus code: {plus_code}\n\n
        """
    return strout

def nearby_dense_community(input_dict: NearbyDenseCommunityInput) -> str:
    """ getting nearby dense community such as (community mall, hotel, school, etc), by location name, radius(in meters)
    return list of location community nearby, name, community type.
    """
    location = input_dict['location_name']
    radius = input_dict['radius']
    
    location_coords = gplace.find_location(location, radius=radius)
    result = gplace.nearby_dense_community(location_coords, radius)
    
    strout = ""
    for r in result:
        # Use .get() to handle missing keys
        address = r.get('vicinity', 'N/A')
        location_types = r.get('types', 'N/A')
        name = r.get('name', 'N/A')
        opening_hours = r.get('opening_hours', 'N/A')
        rating = r.get('rating', 'N/A')
        plus_code = r.get('plus_code', {}).get('global_code', 'N/A')
        
        strout += f"""
        name: {name}\n
        types: {location_types}\n
        """.strip()
    return strout


In [5]:
# gplace_tools.py
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from langchain_core.tools import Tool
from langchain_google_community import GoogleSearchAPIWrapper
from langchain_community.document_loaders import WebBaseLoader

import utils

utils.load_env()

search = GoogleSearchAPIWrapper()

find_place_from_text = tool(find_place_from_text)
nearby_search = tool(nearby_search)
google_search = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)
web_loader = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=WebBaseLoader,
)

tools = [find_place_from_text, nearby_search]

# Create ToolNodes for each tool
tool_node = ToolNode(tools)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
nearby_dense_community({'location_name': 'ลุมพินี เซ็นเตอร์ ลาดพร้าว', 'radius': 8000})

"name: Bangkok\n\n        types: ['locality', 'political']name: Metropoint Bangkok Hotel\n\n        types: ['lodging', 'point_of_interest', 'establishment']name: The Grand Fourwings Convention Hotel\n\n        types: ['lodging', 'point_of_interest', 'establishment']name: Hua Mak Indoor Stadium\n\n        types: ['point_of_interest', 'establishment']name: B2 Bangkok Srinagarindra Boutique & Budget Hotel\n\n        types: ['clothing_store', 'lodging', 'point_of_interest', 'store', 'establishment']name: HappyLand Mansion\n\n        types: ['lodging', 'point_of_interest', 'establishment']name: Bangkok Swimming by Kru Jin\n\n        types: ['point_of_interest', 'establishment']name: Aunchaleena grand Hotel\n\n        types: ['lodging', 'point_of_interest', 'establishment']name: Anda Hotel\n\n        types: ['lodging', 'point_of_interest', 'establishment']name: Grand Mandarin Residence\n\n        types: ['lodging', 'point_of_interest', 'establishment']name: Wallada Place Hotel\n\n        typ

In [8]:
google_search("the pricing of Mimosa Bangkok in Bangkok ").replace("\xa0", "")

'Address: 156/2 Dinso Rd, Sao Chingcha, Phra Nakhon, Bangkok 10200 ; Phone: 02-622-1525 ; Open hours: 10 am – 11 pm daily ; Prices: Most dishes are in the price... For those complaining about the price go sit in the street if you want super cheap. It\'s inside a large shopping mall filled with luxury brands, what do you... If you are looking for quality Mediterranean cuisine in the heart of Bangkok, look no further than Mimosa BKK. ... Don\'t forget to check for happy hour prices so... PRICE RANGE. $5.00 – $86.00 ; CUISINES. Italian, French, Pizza, Mediterranean, Spanish ; Meals. Lunch, Dinner, Brunch, Drinks. 1790 Followers, 6 Following, 111 Posts - Mimosa | Mediterranean Restaurant Bangkok (@mimosabkk) on Instagram: "From the sunny shores of the Mediterranean... Mimosa Cake by Fast Buds is a Sativa-dominant hybrid but thanks to the handpicked terpene profile, its strong Sativa high comes hand-in-hand with a nice. ... Bangkok · Discover Our Stores · PAÑPURI FIRST · Our Stores. EN. EN 